# Hidden Markov Model Example

authors:<br>
Jacob Schreiber [<a href="mailto:jmschreiber91@gmail.com">jmschreiber91@gmail.com</a>]<br>
Nicholas Farn [<a href="mailto:nicholasfarn@gmail.com">nicholasfarn@gmail.com</a>]

A simple example highlighting how to build a model using states, add
transitions, and then run the algorithms, including showing how training
on a sequence improves the probability of the sequence.

In [1]:
import random
from pomegranate import *

random.seed(0)

First we will create the states of the model, one uniform and one normal.

In [2]:
state1 = State( UniformDistribution(0.0, 1.0), name="uniform" )
state2 = State( NormalDistribution(0, 2), name="normal" )

In [3]:
print(state1)

{
    "distribution" : {
        "name" : "UniformDistribution",
        "frozen" : false,
        "class" : "Distribution",
        "parameters" : [
            0.0,
            1.0
        ]
    },
    "name" : "uniform",
    "weight" : 1.0,
    "class" : "State"
}


We will then create the model by creating a HiddenMarkovModel instance. Then we will add the states.

In [4]:
model = HiddenMarkovModel( name="ExampleModel" )
model.add_state( state1 )
model.add_state( state2 )

Now we'll add the start states to the model.

In [5]:
model.add_transition( model.start, state1, 0.5 )
model.add_transition( model.start, state2, 0.5 )

And the transition matrix.

In [6]:
model.add_transition( state1, state1, 0.4 )
model.add_transition( state1, state2, 0.4 )
model.add_transition( state2, state2, 0.4 )
model.add_transition( state2, state1, 0.4 )

Finally the ending states to the model.

In [7]:
model.add_transition( state1, model.end, 0.2 )
model.add_transition( state2, model.end, 0.2 )

To finalize the model, we "bake" it.

In [9]:
model.bake()

New we'll create a sample sequence using our model.

In [10]:
sequence = model.sample()
print(sequence)

[3.528104691935328, 0.8003144167344466]


In [13]:
print(model.log_probability(sequence))

-6.217899048946169


In [14]:
print(model.backward(sequence))

[[-6.4410426  -6.4410426  -6.21789905        -inf]
 [-2.35672581 -2.35672581 -2.13358226        -inf]
 [-1.60943791 -1.60943791        -inf  0.        ]]


Now we'll feed the sequence through a forward algorithm with our model.

In [16]:
print(model.forward( sequence )[ len(sequence), model.end_index ])

-6.21789904895


Next we'll do the same, except with a backwards algorithm.

In [15]:
print(model.backward( sequence )[0,model.start_index])

-6.21789904895


Then we'll feed the sequence again, through a forward-backward algorithm.

In [17]:
trans, ems = model.forward_backward( sequence )
print(trans)
print(ems)

[[ 0.15549349  0.84450651  0.          0.15549349]
 [ 0.          0.          0.          0.84450651]
 [ 1.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
[[ 0.                -inf]
 [-1.86115144 -0.16900283]]


Finally we'll train our model with our example sequence.

In [18]:
print(model.viterbi(sequence))

(-6.38690187828943, [(2, {
    "distribution" : null,
    "name" : "ExampleModel-start",
    "weight" : 1.0,
    "class" : "State"
}), (0, {
    "distribution" : {
        "name" : "NormalDistribution",
        "frozen" : false,
        "class" : "Distribution",
        "parameters" : [
            0.0,
            2.0
        ]
    },
    "name" : "normal",
    "weight" : 1.0,
    "class" : "State"
}), (1, {
    "distribution" : {
        "name" : "UniformDistribution",
        "frozen" : false,
        "class" : "Distribution",
        "parameters" : [
            0.0,
            1.0
        ]
    },
    "name" : "uniform",
    "weight" : 1.0,
    "class" : "State"
}), (3, {
    "distribution" : null,
    "name" : "ExampleModel-end",
    "weight" : 1.0,
    "class" : "State"
})])


In [20]:
sequences = [sequence]
print(model.log_probability(sequence))

-6.217899048946169


Then repeat the algorithms we fed the sequence through before on our improved model.

In [21]:
print(model.fit(sequences))

nan


## HMMs in the CG rich region example

Lets take the simplified example of CG island detection on a sequence of DNA. DNA is made up of the four canonical nucleotides, abbreviated 'A', 'C', 'G', and 'T'. Specific organizations of these nucleotides encode enough information to build you, a human being. One simple region in the genome is called the 'CG' island, where the nucleotides 'C' and 'G' are enriched. Lets compare the predictions of a GMM with the predictions of a HMM, to both understand conceptually the differences between the two, and to see how easy it is to use pomegranate.


In [4]:
from pomegranate import *
import numpy as np
import random

random.seed(0)

Lets start off with an example without tied states and see what happens.

In [5]:
untiedmodel = HiddenMarkovModel( "No Tied States" )

Here we'll define the four states.

In [6]:
background_one = State( DiscreteDistribution({'A': 0.25, 'C':0.25, 'G': 0.25, 'T':0.25 }), name="B1" )
CG_island = State( DiscreteDistribution({'A': 0.1, 'C':0.4, 'G': 0.4, 'T':0.1 }), name="CG" )
background_two = State( DiscreteDistribution({'A': 0.25, 'C':0.25, 'G': 0.25, 'T':0.25 }), name="B2" )
poly_T = State( DiscreteDistribution({'A': 0.1, 'C':0.1, 'G': 0.1, 'T':0.7 }), name="PT" )

Then add the starting transitions.

In [11]:
untiedmodel.add_transition( untiedmodel.start, background_one, 1. )

The transition matrix.

In [7]:
untiedmodel.add_transition( background_one, background_one, 0.9 )
untiedmodel.add_transition( background_one, CG_island, 0.1 )
untiedmodel.add_transition( CG_island, CG_island, 0.8 )
untiedmodel.add_transition( CG_island, background_two, 0.2 )
untiedmodel.add_transition( background_two, background_two, 0.8 )
untiedmodel.add_transition( background_two, poly_T, 0.2 )
untiedmodel.add_transition( poly_T, poly_T, 0.7 )

And finally the ending transitions.

In [8]:
untiedmodel.add_transition( poly_T, untiedmodel.end, 0.3)

Finishing with the method "bake" to finalize the structure of our model.

In [12]:
untiedmodel.bake( verbose=True )

Now let's define the following sequences. Keep in mind training must by done on a list of lists, not on a string in order to allow strings of any length.

In [13]:
sequences = [ numpy.array(list("TAGCACATCGCAGCGCATCACGCGCGCTAGCATATAAGCACGATCAGCACGACTGTTTTT")),
	      numpy.array(list("TAGAATCGCTACATAGACGCGCGCTCGCCGCGCTCGATAAGCTACGAACACGATTTTTTA")),
	      numpy.array(list("GATAGCTACGACTACGCGACTCACGCGCGCGCTCCGCATCAGACACGAATATAGATAAGATATTTTTT")) ]

Let's check our distributions before training our model.

In [14]:
print( "\n".join( "{}: {}".format( state.name, state.distribution ) 
	for state in untiedmodel.states if not state.is_silent() ) )

B1: {
    "class" :"Distribution",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "C" :0.25,
            "A" :0.25,
            "T" :0.25,
            "G" :0.25
        }
    ],
    "frozen" :false
}
B2: {
    "class" :"Distribution",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "C" :0.25,
            "A" :0.25,
            "T" :0.25,
            "G" :0.25
        }
    ],
    "frozen" :false
}
CG: {
    "class" :"Distribution",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "C" :0.4,
            "A" :0.1,
            "T" :0.1,
            "G" :0.4
        }
    ],
    "frozen" :false
}
PT: {
    "class" :"Distribution",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "C" :0.1,
            "A" :0.1,
            "T" :0.7,
            "G" :0.1
        }
    ],
    "frozen" :false
}


Now lets train our model

In [15]:
untiedmodel.fit( sequences, stop_threshold=0.01 )

20.623102184194636

And check our new deistributions after training.

In [16]:
print("\n".join( "{}: {}".format( state.name, state.distribution ) 
	for state in untiedmodel.states if not state.is_silent() ) )

B1: {
    "class" :"Distribution",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "C" :0.2920431428628609,
            "A" :0.31639790507943644,
            "T" :0.1796452508025389,
            "G" :0.21191370125516354
        }
    ],
    "frozen" :false
}
B2: {
    "class" :"Distribution",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "C" :0.22068400964255322,
            "A" :0.41787296061641543,
            "T" :0.16585625898436449,
            "G" :0.19558677075666675
        }
    ],
    "frozen" :false
}
CG: {
    "class" :"Distribution",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "C" :0.5058121809386025,
            "A" :0.061102410527201514,
            "T" :0.09464208215707064,
            "G" :0.33844332637712554
        }
    ],
    "frozen" :false
}
PT: {
    "class" :"Distribution",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "C" :4.501540436

In [19]:
seq = list('CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC')

In [20]:
print(untiedmodel.predict(seq))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
